# Extract nuclear measurements

Scripts to extract nuclear measurements from segmented nuclei

**Inputs**:

1) Pandas dataframe fileList.csv (or fileList_wormMasks.csv) with absolute paths to raw .nd2 files and denoised .tif files in columns named raw_filepath and denoised_filepath.
An 'id' column has a unique id for each image. Other metadata columns can also be present.
If using worm region masks, it must have a column called worm_regions_all with ';' separated list of worm region types.

2) output_path is the location of the file produced by this script

**Outputs**:

nuclear measurements (.csv files) in output_path/nuclei/

intensity measurements for nuclei with arrays of intensity/distance from middle slice of each nuclear mask (.pkl files) in as well as other nuclear measurments and data are in output_path/dist/ 

qc plots of individual masked nuclei (cropped_nuclei_XXX.pdf) in output_path/qc/


### Settings you might need to change

output_path - create a directory for the analysis. results will be stored in a protein/strain/date structure same as in the raw_input_path.

path_type - “server”, “mac” or “wsl” so I can switch between working on server or with izbkingston mounted on mac/pc. (for PC izbkingston needs to be mounted with sshfs as /mnt/izbkingston/ from within WSL). Since this script works best with gpu, it will almost always be "server"

Set the channel number (as it is in the orignal image) for nuclear marker (nucChannel) and for spots (spotChannel). If there is no additional nuclear marker channel, set it to the same as the spotChannel

use_worm_masks - True if you presegmented particular worm regions which you later want to filter your nuclei by (this determines whether you use fileList.csv or fileList_wormMasks.csv)

maxradius - maximum number of pixel-lag used in autocorrelation. (12 pixels is about 3x PSF in images with pixel diameter of 65nm), but longer lags might be worth investigaing.


In [6]:
from skimage.measure import regionprops_table, regionprops
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import edt
import os
import tqdm
from matplotlib_scalebar.scalebar import ScaleBar
from bioio import BioImage
import bioio_nd2
import bioio_tifffile
from bioio.writers import OmeTiffWriter
import scipy.stats as stats
from scipy import optimize
from skimage import morphology
from convert_paths import correct_path, correct_save_path, correct_loaded_path


pd.set_option('display.max_columns', None)

## Input settings

In [ ]:
nucChannel = 0 # red emerin rings if present, otherwise set to the same as spotChannel
spotChannel = 0 # green spots
path_type="server" # "server" or "mac" or "wsl"
use_worm_masks = True # necessary to use worm_masks to subset nuclei from particular regions of the image
maxradius = 12 # maximum radius for autocorrelation (in pixels)


#output_path = correct_path('/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Kalyan/2025-25-02_bet1-mSG_wPM1353')
#output_path = correct_path('/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Kalyan/2025-04-03_bet1-mSG_wPM1353')
#output_path = correct_path('/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Kalyan/2025-10-05_bet1-mSG_wPM1353')
output_path = '/mnt/external.data/MeisterLab/jsemple/demo_VIBE/results/2025-10-05_bet1-mSG_wPM1353'
output_path = correct_path(output_path, path_type)



if use_worm_masks:
    df = pd.read_csv(os.path.join(output_path,'fileList_wormMasks.csv'))
    df = correct_loaded_path(df, path_type)
else:
    df = pd.read_csv(os.path.join(output_path,'fileList.csv'))
    df = correct_loaded_path(df, path_type)

df.head()

,filename,date,protein,strain,treatment,worm_id,id,raw_filepath,denoised_filepath,worm_regions_all
0,2025_10_05_wPM1353_HS_001,2025-10-05,bet1-mSG,wPM1353,HS,1,bet1-mSG_2025-10-05_2025_10_05_wPM1353_HS_001,/mnt/external.data/MeisterLab/jsemple/demo_VIB...,/mnt/external.data/MeisterLab/jsemple/demo_VIB...,head;body_other
1,2025_10_05_wPM1353_nHS_001,2025-10-05,bet1-mSG,wPM1353,nHS,1,bet1-mSG_2025-10-05_2025_10_05_wPM1353_nHS_001,/mnt/external.data/MeisterLab/jsemple/demo_VIB...,/mnt/external.data/MeisterLab/jsemple/demo_VIB...,head;body_other


In [8]:
if not os.path.exists(os.path.join(output_path,"qc")):
    os.makedirs(os.path.join(output_path,"qc"))

if not os.path.exists(os.path.join(output_path,"nuclei")):
    os.makedirs(os.path.join(output_path,"nuclei"))

if not os.path.exists(os.path.join(output_path,"dist")):
    os.makedirs(os.path.join(output_path,"dist"))

## Functions for nuclear segmentation qc

In [9]:

def plot_qc_nuclei_crop(df, index, df_region_props, img, t=0, display = False, seed=1):
    '''Plot a cropped region of a random sample of 10 nuclei from each image'''
    nb_nuc = 10
    np.random.seed(seed)
    indices_to_sample = np.random.choice(range(len(df_region_props)),size = nb_nuc,replace = False)
    # sort indices in descending order of area

    widths=[df_region_props['image'][i].shape[1] for i in indices_to_sample]

    if spotChannel != nucChannel:
        fig, axs = plt.subplots(nrows = 2, ncols = nb_nuc, figsize = (15,5),dpi = 250, 
                                sharex=False, sharey=False, width_ratios=widths)
        
        for i,sample in enumerate(indices_to_sample):
            intensity_image = df_region_props['intensity_image'][sample][:,:,:,spotChannel] #show first spot channel
            image = df_region_props['image'][sample]
            mx = np.ma.masked_array(intensity_image,mask = ~image)
            z_height = image.shape[0] 
            axs[0,i].imshow(mx[int(z_height/2)])
            axs[0,i].spines['top'].set_visible(False)
            axs[0,i].spines['right'].set_visible(False)
            axs[0,i].spines['bottom'].set_visible(False)
            axs[0,i].spines['left'].set_visible(False)
            axs[0,i].get_xaxis().set_ticks([])
            axs[0,i].get_yaxis().set_ticks([])
        
        for i,sample in enumerate(indices_to_sample):
            intensity_image = df_region_props['intensity_image'][sample][:,:,:,nucChannel] #show second nuclear channel
            image = df_region_props['image'][sample]
            mx = np.ma.masked_array(intensity_image,mask = ~image)
            z_height = image.shape[0]
            axs[1,i].imshow(mx[int(z_height/2)])
            axs[1,i].spines['top'].set_visible(False)
            axs[1,i].spines['right'].set_visible(False)
            axs[1,i].spines['bottom'].set_visible(False)
            axs[1,i].spines['left'].set_visible(False)
            axs[1,i].get_xaxis().set_ticks([])
            axs[1,i].get_yaxis().set_ticks([])
    else:
        fig, axs = plt.subplots(nrows = 1, ncols = nb_nuc, figsize = (7.5,5),dpi = 250, 
                            sharex=False, sharey=False, width_ratios=widths)

        for i,sample in enumerate(indices_to_sample):
            intensity_image = df_region_props['intensity_image'][sample][:,:,:,spotChannel] #show first spot channel
            image = df_region_props['image'][sample]
            mx = np.ma.masked_array(intensity_image,mask = ~image)
            z_height = image.shape[0] 
            axs[i].imshow(mx[int(z_height/2)])
            axs[i].spines['top'].set_visible(False)
            axs[i].spines['right'].set_visible(False)
            axs[i].spines['bottom'].set_visible(False)
            axs[i].spines['left'].set_visible(False)
            axs[i].get_xaxis().set_ticks([])
            axs[i].get_yaxis().set_ticks([])

    fig.suptitle(f'Cropped nuclei {df.id.iloc[index]}', fontsize=16)

    if i == nb_nuc-1:
        scalebar = ScaleBar(0.065, "um", length_fraction=1, box_alpha=0.7,color='black',location='lower right',height_fraction = 0.05,border_pad =-1)
        if spotChannel != nucChannel:
            axs[1,i].add_artist(scalebar)
        else:
            axs[i].add_artist(scalebar)

    #plt.tight_layout()
    fig.savefig(os.path.join(output_path,'qc/cropped_nuclei_'+df.id.iloc[index]+'_t'+'{:02d}'.format(t)+'.pdf'))
    if display == False:
        plt.close()
    else:
        plt.show()


def plot_single_nucleus_crop(df, index, df_region_props, nuc_index, img):
    '''Plot a cropped region of a particular nucleus'''
    if spotChannel != nucChannel:
        fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (3,1.5),dpi = 250, sharey=True)
    else:
        fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (1.5,1.5),dpi = 250, sharey=True)
    fig.suptitle(f'{df.id.iloc[index]}', fontsize=6)

    intensity_image = df_region_props['intensity_image'][nuc_index][:,:,:,spotChannel] #show first spot channel
    image = df_region_props['image'][nuc_index]
    mx = np.ma.masked_array(intensity_image, mask = ~image)
    z_height = image.shape[0] 
    axs[0].imshow(mx[int(z_height/2)])
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['bottom'].set_visible(False)
    axs[0].spines['left'].set_visible(False)
    axs[0].get_xaxis().set_ticks([])
    axs[0].get_yaxis().set_ticks([])

    if spotChannel != nucChannel:
        intensity_image = df_region_props['intensity_image'][nuc_index][:,:,:,nucChannel] #show second nuclear channel
        image = df_region_props['image'][nuc_index]
        mx = np.ma.masked_array(intensity_image, mask = ~image)
        z_height = image.shape[0]
        axs[1].imshow(mx[int(z_height/2)])
        axs[1].spines['top'].set_visible(False)
        axs[1].spines['right'].set_visible(False)
        axs[1].spines['bottom'].set_visible(False)
        axs[1].spines['left'].set_visible(False)
        axs[1].get_xaxis().set_ticks([])
        axs[1].get_yaxis().set_ticks([])


    scalebar = ScaleBar(0.065, "um", length_fraction=1, box_alpha=0.7,color='black',location='lower right',height_fraction = 0.05,border_pad =-1)
    if spotChannel != nucChannel:
        axs[1].add_artist(scalebar)
    else:
        axs[0].add_artist(scalebar) 

    plt.show()



## Functions for autocorrelation

In [10]:
def get_nuc_background_image_mask(img, i, df_region_props, spotChannel):
    '''
    Uses bounding box for nucleus and the original image to extract
    intensity values and mask for background pixels close to the nucleus.
    '''
    z1 = df_region_props['bbox-0'].iloc[i]
    z2 = df_region_props['bbox-3'].iloc[i]
    y1 = df_region_props['bbox-1'].iloc[i]
    y2 = df_region_props['bbox-4'].iloc[i]
    x1 = df_region_props['bbox-2'].iloc[i]
    x2 = df_region_props['bbox-5'].iloc[i]
    #print(z1,z2,y1,y2,x1,x2)
    if z2==z1:
        mskd = np.ma.masked_array(img[z1,y1:y2,x1:x2,spotChannel], mask = df_region_props['image'].iloc[i],fill_value=0)
    else:
        mskd = np.ma.masked_array(img[z1:z2,y1:y2,x1:x2,spotChannel], mask = df_region_props['image'].iloc[i],fill_value=0)
    bg_image = mskd.data
    bg_image[mskd.mask] = 0 # remove unwanted data to save space
    # note that in masked arrays True is invalid data, so need to reverse logic of mask
    bg_mask = np.logical_not(mskd.mask)
    return(bg_image, bg_mask)

In [11]:
def center_image_values(image,image_mask):
    '''
    Centers image values around the mean, taking
    into account a mask, and zeros the masked region.
    '''
    norm_image=image-image[image_mask].mean()
    norm_image[np.logical_not(image_mask)] = 0
    return(norm_image)



def get_autocorrelation_2d(img,img_mask,maxr=maxradius):
    '''
    Calculates autocorrelation function for a single
    2-3d image with a mask, according to
    Munschi et al. 2025, with additional normalisation
    steps.
    '''
    xdim = img.shape[2]
    ydim = img.shape[1]
    zdim = img.shape[0]
    norm_img = center_image_values(img,img_mask)
    yautocorr = np.zeros((ydim))
    xz_combinations=0
    for x in range(0,xdim):
        for z in range(0,zdim):
            validPixels=np.sum(norm_img[z,:,x]!=0)
            if validPixels>1:
                result = np.correlate(norm_img[z,:,x],norm_img[z,:,x],mode='full')
                normResult = result[result.size//2:]/(validPixels*(validPixels-1)/2)
                yautocorr=yautocorr + normResult
                xz_combinations+=1
    if xz_combinations>0:
        yautocorr = yautocorr/xz_combinations
    else:
        yautocorr = np.zeros((ydim))

    
    xautocorr = np.zeros((xdim))
    yz_combinations=0
    for y in range(0,ydim):
        for z in range(0,zdim):
            validPixels=np.sum(norm_img[z,y,:]!=0)
            if validPixels>1:  
                result = np.correlate(norm_img[z,y,:],norm_img[z,y,:],mode='full')
                normResult = result[result.size//2:]/(validPixels*(validPixels-1)/2)
                xautocorr=xautocorr + normResult
                yz_combinations+=1
    if yz_combinations>0:
        xautocorr = xautocorr/yz_combinations
    else:
        xautocorr = np.zeros((xdim))
    # pad the smaller autocorr to match the size of the larger one
    if(xdim>=ydim):
        yautocorr = np.pad(yautocorr,(0,xdim-ydim))
    elif(ydim>xdim):
        xautocorr = np.pad(xautocorr,(0,ydim-xdim))
    # average the two autocorrs
    autocorr = (xautocorr+yautocorr)/2
    if(len(autocorr)<(maxr+1)): #ensure autocorr is long enough
        autocorr = np.pad(autocorr,(0,(maxr+1)-len(autocorr)))
    # normalise by variance 
    varNorm = autocorr[0:(maxr+1)]/autocorr[0]
    return(varNorm)


def exponential_decay(x, a, b,c):
    '''
    Exponenetial decay function for autocorrelation
    according to Munschi et al. 2025.
    '''                           
    return a + b * np.exp(-c * x)      


def fit_acf(autocorr):
    '''
    Fits parameters of an exponential decay function 
    to the autocorrelation values. Used for single
    nuclei.
    '''
    x=range(0,autocorr.size)
    initialguess = [0.2, 0.8, 0.5]
    autocorr = autocorr.astype(np.float64)
    try:
        fit, covariance = optimize.curve_fit(           
            exponential_decay,                                     
            x,   
            autocorr,  #np.insert(autocorr,0,1),
            initialguess,
            bounds=([0., 0., 0.], [1., 1., 100.]),
            method='trf', 
            loss='linear')
        if np.nan not in fit and np.all(np.isfinite(fit)):
            rmse = np.sqrt(np.mean((autocorr - exponential_decay(x, *fit))**2))
            conditionNumber = np.linalg.cond(covariance)
        else:
            rmse = np.nan 
            conditionNumber = np.nan
    except RuntimeError:
        fit = [0,0,0] #[np.nan, np.nan, np.nan]
        covariance = np.zeros((3,3)) 
        rmse = np.nan 
        conditionNumber = np.nan
    return(fit, covariance, rmse, conditionNumber)


def correlation_length(fit):
    '''
    Calculates correlation length according to
    Munschi et al. 2025.
    '''
    if np.nan not in fit:
        c=fit[2]
        x0=1
        lam=x0+np.log(2)/c
    else: 
        lam = 0 #np.nan
    return(lam)


def correlation_error(fit,cov):
    '''
    Calculates correlation length error according to
    Munschi et al. 2025.
    '''
    if np.nan not in fit:
        c=fit[2]
        sigma_c = np.sqrt(cov[2,2])
        lam = correlation_length(fit)
        lam_error = lam*sigma_c/c
    else:
        lam_error = 0 #np.nan
    return(lam_error)




def get_autocorrelation_length(img,mask):
    '''
    Calculates autocorelation length from 
    a 2d or 3d image and mask. 
    '''
    ac = get_autocorrelation_2d(img, mask, maxr=maxradius)
    fit, cov, rmse, conditionNumber = fit_acf(ac)
    fiterr = np.sqrt(np.diag(cov))
    if np.nan not in fit:
        ac_length = correlation_length(fit)
        ac_error = correlation_error(fit, cov) 
        if(ac_length<np.abs(ac_error)):
            ac_length = 0#np.nan
            ac_error = 0#np.nan
    else:
        ac_length = 0#np.nan
        ac_error = 0#np.nan
    return(ac, ac_length, ac_error, rmse, conditionNumber, fit, fiterr)



In [ ]:
# Function for trouble shooting - plots ACF along with image of spot channel
# when given the nuclei table along with an index number
def plot_one_nucleus_acf(df, i):
    '''
    Plots the autocorrelation function for a specific row in the DataFrame
    and displays the corresponding image in a second subplot.
    '''
    img = df['intensity_image'].iloc[i][:, :, :, spotChannel]
    zdim=img.shape[0]
    img_mask = df['image'].iloc[i]
    autocorr = get_autocorrelation_2d(img, img_mask, maxr=maxradius)
    fit, covariance, rmse, conditionNumber = fit_acf(autocorr)
    fiterr = np.sqrt(np.diag(covariance))
    x = range(0, autocorr.size)

    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))


    # Plot the autocorrelation function in the first subplot
    axes[0].plot(x, autocorr, 'o', label='data')
    try:
        axes[0].plot(x, exponential_decay(x, *fit), label='fit')
    except:
        pass
    axes[0].legend()
    axes[0].set_title('Autocorrelation '+str(i)+' '+df['quality'].iloc[i]+' '+df['confusion'].iloc[i])
    axes[0].set_xlabel('Distance')
    axes[0].set_ylabel('Autocorrelation')
    axes[0].text(0.5, 0.5, 
                 f'lambda = {correlation_length(fit):.2f} +/- {correlation_error(fit, covariance):.2f}', 
                 transform=axes[0].transAxes)
    axes[0].text(0.5, 0.6, 'fit: a={:.3g}±{:.3g},\nb={:.3g}±{:.3g},\nc={:.3g}±{:.3g}'.format(
                     fit[0], fiterr[0], fit[1], fiterr[1], fit[2], fiterr[2]), 
                 transform=axes[0].transAxes)
    axes[0].text(0.5, 0.7, 'RMSE = {:.3g}'.format(rmse), transform=axes[0].transAxes)
    axes[0].text(0.5, 0.8, 'conditionNumber = {:.3g}'.format(conditionNumber), transform=axes[0].transAxes)
    # Display the image in the second subplot
    axes[1].imshow(img[int(zdim/2),:,:], cmap='gray')
    axes[1].set_title('Spots Channel')
    axes[1].axis('off')

    # Show the plots
    plt.tight_layout()
    plt.show()

## Function to extract nuclei measurements

In [13]:

def run_dist_analysis(indices,df, use_worm_masks = False):
    '''Run the distance analysis on all images in the dataframe'''
    for index in tqdm.tqdm(indices):
        #print(df.iloc[index].raw_filepath)
        img_5d = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
        # calculate anisotropy from raw image metadata
        ZvX = np.round(img_5d.physical_pixel_sizes.Z/img_5d.physical_pixel_sizes.X,0)
        df_nuclei_all = pd.DataFrame()
        for t in tqdm.tqdm(range(img_5d.dims.T)):
            #print(t)
            df_nuclei = pd.DataFrame()
            img_all = img_5d.get_image_data("ZYXC", T=t)

            nuc_masks = BioImage(os.path.join(output_path,'segmentation',df.id.iloc[index]+'_t'+'{:02d}'.format(t)+'.tif'), reader=bioio_tifffile.Reader)
            nuc_masks = nuc_masks.get_image_data("ZYX", T=0, C=0)

            if use_worm_masks:
                worm_regions=df['worm_regions_all'].iloc[index].split(';')
            else:
                worm_regions = ['all']
            for worm_region in worm_regions:
                # subset nuclear masks with masks of worm region
                worm_mask_path = os.path.join(output_path, 'worm_masks', df.id.iloc[index] + '_' + worm_region + '.tif')
                if use_worm_masks and os.path.exists(worm_mask_path):
                    print("measuring nuclei in "+ worm_region)
                    worm_mask = BioImage(worm_mask_path, reader=bioio_tifffile.Reader)
                    worm_mask = worm_mask.get_image_data("YX",Z=0, T=0, C=0)
                    masks = nuc_masks * worm_mask # subset nuclear masks with worm region
                else:
                    print("measuring all nuclei")
                    masks = nuc_masks.copy()
    
                img = img_all.copy() # otherwise overlapping masks give empty intensity images for nuclei in overlap
                masks=morphology.remove_small_objects(masks, min_size=2000) # to remove small objects created by masks
                
                df_region_props = regionprops_table(masks,img, properties = ['label', 'bbox', 'area','centroid','MajorAxisLength','solidity','image','intensity_image'])
                df_region_props = pd.DataFrame(df_region_props)

                if len(df_region_props)>=10:
                    plot_qc_nuclei_crop(df, index, df_region_props, img, t=t, display = False) 
                
                if len(df_region_props)>=1:
                    for i in tqdm.tqdm(range(len(df_region_props))):
                        df_nuclei_temp = pd.DataFrame()

                        intensity_image_spots = df_region_props['intensity_image'].iloc[i][:,:,:,spotChannel] #show spot channel
                        # if np.all(intensity_image_spots == 0):
                        #     print('EMPTY SPOT CHANNEL!!')
                        #     print(df.filename.iloc[index]+str(i)+ worm_region)
                        #     continue
                        if spotChannel != nucChannel:
                            intensity_image_nuclei = df_region_props['intensity_image'].iloc[i][:,:,:,nucChannel] #show nuclear ring channel

                        image = df_region_props['image'][i]  # binary 3d mask

                        # Extract the intensity per distance
                        mx_spots = np.ma.masked_array(intensity_image_spots, mask = ~image) # 3d masked spot channel
                        if spotChannel != nucChannel:
                            mx_nuclei = np.ma.masked_array(intensity_image_nuclei,mask = ~image) # 3d masked nuclear ring channel
                        mx_mask = np.ma.masked_array(image,mask = ~image)  # 3d masked binary mask

                        z_height = image.shape[0]

                        slice_spots = mx_spots[int(z_height/2)]
                        if spotChannel != nucChannel:
                            slice_nuclei = mx_nuclei[int(z_height/2)]
                        slice_mask = mx_mask[int(z_height/2)]

                        slice_mask_edt = edt.edt(slice_mask)
                        slice_mask_edt = np.ma.masked_array(slice_mask_edt, mask = ~(slice_mask_edt>0)) 

                        results = regionprops_table(slice_mask_edt.astype('int'),slice_spots,properties=['label','intensity_mean'])
                        intensity_dist_spots = results['intensity_mean']

                        if spotChannel != nucChannel:
                            results = regionprops_table(slice_mask_edt.astype('int'),slice_nuclei,properties=['label','intensity_mean'])
                            intensity_dist_nuclei = results['intensity_mean']

                        dist = results['label']

                        #background pixels for from bounding box of nucleus
                        bg_image, bg_mask = get_nuc_background_image_mask(img, i, df_region_props, spotChannel)

                        df_nuclei_temp =  pd.DataFrame([df_region_props.iloc[i]])
                        df_nuclei_temp.rename(columns = {"centroid-0": "centroid_z", "centroid-1": "centroid_y", "centroid-2": "centroid_x",
                                                        "MajorAxisLength": "major_axis_length"}, inplace = True)
                        df_nuclei_temp['worm_region'] = worm_region
                        df_nuclei_temp['intensity_background'] = [bg_image]
                        df_nuclei_temp['mask_background'] = [bg_mask]
                        df_nuclei_temp['bb_dimZ']  = [mx_spots.shape[0]]
                        df_nuclei_temp['bb_dimY']  = [mx_spots.shape[1]]
                        df_nuclei_temp['bb_dimX']  = [mx_spots.shape[2]]
                        df_nuclei_temp['mean'] = np.ma.mean(mx_spots)
                        df_nuclei_temp['median'] = np.ma.median(mx_spots)
                        df_nuclei_temp['std']=  np.ma.std(mx_spots)
                        df_nuclei_temp['sum']= np.ma.sum(mx_spots)
                        df_nuclei_temp['variance']= np.ma.var(mx_spots)
                        df_nuclei_temp['max'] = np.ma.max(mx_spots)
                        df_nuclei_temp['min'] = np.ma.min(mx_spots)
                        df_nuclei_temp['volume'] = np.sum(np.invert(mx_spots.mask))
                        df_nuclei_temp['mean_background'] = bg_image[bg_mask].mean()
                        df_nuclei_temp['std_background'] = bg_image[bg_mask].std()
                        df_nuclei_temp['sum_background'] = bg_image[bg_mask].sum()
                        df_nuclei_temp['volume_background'] = bg_mask.sum()
                        df_nuclei_temp['id'] = df.id.iloc[index]
                        df_nuclei_temp['timepoint'] = t
                        df_nuclei_temp['intensity_dist_spots'] = [intensity_dist_spots] # this is the spot channel but not actual detected spots
                        if spotChannel != nucChannel:
                            df_nuclei_temp['intensity_dist_nuclei'] = [intensity_dist_nuclei]  # this is the emerin ring channel intensity on central slice
                        df_nuclei_temp['intensity_dist'] = [dist]  # this is the distance from the edge of the nucleus
                        df_nuclei_temp['zproj_spots'] = [np.max(intensity_image_spots[:,:,:], axis = 0)]
                        if spotChannel != nucChannel:
                            df_nuclei_temp['zproj_nuclei'] = [np.max(intensity_image_nuclei[:,:,:], axis = 0)]
                        df_nuclei_temp['zproj_background'] = [np.max(bg_image[:,:,:], axis = 0)]
                        df_nuclei_temp['anisotropy'] = ZvX
                        df_nuclei_temp['pixelSize'] = img_5d.physical_pixel_sizes.X

                        # get autocorrelation data
                        ac, ac_length, ac_error, rmse, conditionNumber, fit, fiterr = get_autocorrelation_length(df_nuclei_temp['intensity_image'].iloc[0][:,:,:,spotChannel],df_nuclei_temp['image'].iloc[0])
                        df_nuclei_temp['spot_ACF'] = [ac]
                        df_nuclei_temp['spot_ac_length'] = ac_length
                        df_nuclei_temp['spot_ac_error'] = ac_error
                        df_nuclei_temp['spot_ac_rmse'] = rmse
                        df_nuclei_temp['spot_ac_conditionNumber'] = conditionNumber
                        df_nuclei_temp['spot_ac_fittedParams'] = [fit]
                        df_nuclei_temp['spot_ac_fittedParams_err'] = [fiterr]

                        if spotChannel != nucChannel:
                            ac, ac_length, ac_error, rmse, conditionNumber, fit, fiterr = get_autocorrelation_length(df_nuclei_temp['intensity_image'].iloc[0][:,:,:,nucChannel],df_nuclei_temp['image'].iloc[0])
                            df_nuclei_temp['nuc_ACF'] = [ac]
                            df_nuclei_temp['nuc_ac_length'] = ac_length
                            df_nuclei_temp['nuc_ac_error'] = ac_error
                            df_nuclei_temp['nuc_ac_rmse'] = rmse
                            df_nuclei_temp['nuc_ac_conditionNumber'] = conditionNumber
                            df_nuclei_temp['nuc_ac_fittedParams'] = [fit]
                            df_nuclei_temp['nuc_ac_fittedParams_err'] = [fiterr]

                        ac, ac_length, ac_error, rmse, conditionNumber, fit, fiterr = get_autocorrelation_length(df_nuclei_temp['intensity_background'].iloc[0],df_nuclei_temp['mask_background'].iloc[0])
                        df_nuclei_temp['bg_ACF'] = [ac]
                        df_nuclei_temp['bg_ac_length'] = ac_length
                        df_nuclei_temp['bg_ac_error'] =  ac_error
                        df_nuclei_temp['bg_ac_rmse'] = rmse
                        df_nuclei_temp['bg_ac_conditionNumber'] = conditionNumber
                        df_nuclei_temp['bg_ac_fittedParams'] = [fit]
                        df_nuclei_temp['bg_ac_fittedParams_err'] = [fiterr]

                        df_nuclei = pd.concat([df_nuclei,df_nuclei_temp])

                    df_nuclei_all = pd.concat([df_nuclei_all,df_nuclei])

        if(len(df_nuclei_all) > 0):
            # merge with metadata
            df_for_csv = df.merge(df_nuclei_all,on='id',how='right')
            # save as pickle because has array stored in Dataframe
            df_for_csv.reset_index(drop=True, inplace=True)
            df_for_csv.to_pickle(os.path.join(output_path,'dist',df.id.iloc[index]+'.pkl')) # Back up the DF for this FOV

            # save simple data as table 
            simple_columns = [col for col in df_for_csv.columns if df_for_csv[col].apply(lambda x: not isinstance(x, (list, np.ndarray))).all()]
            df_for_csv[simple_columns].to_csv(os.path.join(output_path,'nuclei',df.id.iloc[index]+'.csv'), index=False)

## Functions to gather results into single file

In [14]:
def collect_nuclear_segmentation_data(indices, df, suffix = 'v001'):
    '''Collects nuclear intensity and intensity vs distance data for all nuclei in the dataset'''
    df_nuclei = pd.DataFrame()
    for index in tqdm.tqdm(indices):
        if os.path.exists(os.path.join(output_path,'nuclei',df.id.iloc[index]+'.csv')):
            df_tmp = pd.read_csv(os.path.join(output_path,'nuclei',df.id.iloc[index]+'.csv'))
            df_nuclei = pd.concat([df_nuclei,df_tmp])
    df_nuclei.to_csv(os.path.join(output_path,'nuclei_analysis_'+suffix+'.csv'),index=False)



def collect_nuclear_distance_data(indices, df, suffix = 'v001'):
    '''Collects nuclear intensity and intensity vs distance data for all nuclei in the dataset'''
    df_dist = pd.DataFrame()
    for index in tqdm.tqdm(indices):
        if os.path.exists(os.path.join(output_path,'dist',df.id.iloc[index]+'.pkl')):
            df_tmp = pd.read_pickle(os.path.join(output_path,'dist',df.id.iloc[index]+'.pkl'))
            df_dist = pd.concat([df_dist,df_tmp])
    df_dist.to_pickle(os.path.join(output_path,'dist_analysis_'+suffix+'.pkl'))




## Running the analysis for nuclear segmentation

In [15]:
# run analysis to get measurements for all nuclei in each image
indices=range(0,len(df))
run_dist_analysis(indices, df, use_worm_masks = use_worm_masks)

# collect data from all images into single file
indices=range(0,len(df))
collect_nuclear_segmentation_data(indices, df, suffix = 'v001')
collect_nuclear_distance_data(indices, df, suffix = 'v001')

  0%|          | 0/2 [00:00<?, ?it/s]/home/jsemple/miniforge3/envs/lhcellpose2/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


measuring nuclei in head










































































































































100%|██████████| 265/265 [00:22<00:00, 11.56it/s]


measuring nuclei in body_other




 50%|█████     | 1/2 [01:04<01:04, 64.44s/it]/home/jsemple/miniforge3/envs/lhcellpose2/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


measuring nuclei in head




















































































100%|██████████| 104/104 [00:14<00:00,  6.96it/s]


measuring nuclei in body_other






100%|██████████| 2/2 [00:00<00:00,  6.04it/s]
